In [3]:
import pandas as pd
import re

import spacy

In [4]:
nlp = spacy.load('pl_core_news_lg')

In [5]:
def split_text(text: str) -> list:
    text = text.replace('\n\n', '<p>')
    text = ' '.join(text.split())
    text = text.replace('<p>', '\n')
    # text = text.lower()
    return text

In [6]:
df_path = '../data/parsed/kiids_parsed.parquet'

In [7]:
df = pd.read_parquet(df_path)

In [4]:
df_practial_info = df['informacje praktyczne']
df_practial_info = df_practial_info[~df_practial_info.isna()]

In [5]:
df_practial_info

0      INFORMACJE PRAKTYCZNE\nNiniejszy dokument opis...
1      Informacje praktyczne \nDepozytariusz: Bank Pe...
2                               INFORMACJE PRAKTYCZNE \n
3      Informacje praktyczne \nSubfundusz został utwo...
5      Informacje praktyczne \nDepozytariusz: Bank Pe...
                             ...                        
396    Informacje praktyczne \nSubfundusz został utwo...
397    Informacje praktyczne \nDepozytariusz: Bank Pe...
398                             INFORMACJE PRAKTYCZNE \n
399    Informacje praktyczne \nKluczowe informacje dl...
400    Informacje praktyczne\nKluczowe informacje dla...
Name: informacje praktyczne, Length: 391, dtype: object

In [7]:
df_practial_info = df_practial_info.apply(split_text)

In [8]:
df_practial_info

0      INFORMACJE PRAKTYCZNE Niniejszy dokument opisu...
1      Informacje praktyczne Depozytariusz: Bank Peka...
2                                  INFORMACJE PRAKTYCZNE
3      Informacje praktyczne Subfundusz został utworz...
5      Informacje praktyczne Depozytariusz: Bank Peka...
                             ...                        
396    Informacje praktyczne Subfundusz został utworz...
397    Informacje praktyczne Depozytariusz: Bank Peka...
398                                INFORMACJE PRAKTYCZNE
399    Informacje praktyczne Kluczowe informacje dla ...
400    Informacje praktyczne Kluczowe informacje dla ...
Name: informacje praktyczne, Length: 391, dtype: object

# Depositary

In [47]:
depo_regex = re.compile('depozytariusz(em.*jest)?:?\s*(.*?)(S\.A)?(\.|,|;)', re.IGNORECASE)
not_depo_regex = re.compile('(niezgodne.*)|(to\s)')

In [48]:
def get_depo(text: str) -> str or None:
    regex = depo_regex.search(text)
    if regex is None:
        return None
    else:     
        if not_depo_regex.search(regex.group(2)):
            return None
        
        if regex.group(3) is not None:
            return regex.group(2) + regex.group(3)
        else:
            return regex.group(2)

In [49]:
df_depo_re = df_practial_info.apply(get_depo)

In [50]:
df_depo_re[~df_depo_re.isna()].unique()

array(['Bank Handlowy w Warszawie SA', 'Bank Pekao S.A',
       'Depozytariuszem funduszu jest mBank S.A',
       'Deutsche Bank Polska Akcyjna z siedzibą w Warszawie',
       'ING Bank Śląski S.A', 'PKO Bank Polski S.A',
       'Deutsche Bank Polska Spółka Akcyjna z siedzibą w Warszawie',
       'mBank S.A', 'BNP Paribas Bank Polska S.A',
       'określona w tych umowach lub zasadach',
       'Bank Handlowy w Warszawie S.A',
       'Bank Polska Kasa Opieki Spółka Akcyjna z siedzibą w Warszawie'],
      dtype=object)


# KRS

In [76]:
krs_regex = re.compile('(krs:?\s*)([0-9]{9})', re.IGNORECASE)

In [86]:
def get_krs(text: str) -> str or None:
    regex = krs_regex.search(text)
    if regex is None:
        return None
    else:
        return regex.group(2)

In [87]:
krs = df_practial_info.apply(get_krs)

In [88]:
krs[~krs.isna()]

9      000003112
134    000003112
300    000003112
Name: informacje praktyczne, dtype: object

# NIP

In [131]:
nip_regex = re.compile('(nip:?\s*)([0-9\-]*)(\s|\.|,)', re.IGNORECASE)

In [135]:
def get_nip(text: str) -> str:
    regex = nip_regex.search(text)
    if regex is None:
        return None
    else:
        return regex.group(2)

In [136]:
nip = df_practial_info.apply(get_nip)

In [137]:
nip[~nip.isna()]

9      526-02-10-808
134    526-02-10-808
300    526-02-10-808
Name: informacje praktyczne, dtype: object

# SZIEDZIBA_TOWARZYSTWA

In [228]:
address_regex = re.compile('(ul\.\s\w+\s\d{1,3},?\s\d{2}-\d{3}\s\w+)')

In [232]:
def get_address(text: str) -> str:
    regex = address_regex.search(text)
    if regex is None:
        return None
    else:
        return regex.group(1)

In [233]:
addressess = df_practial_info.apply(get_address)

In [234]:
addressess[~addressess.isna()]

9       ul. Grzybowskiej 78, 00-844 w
28     ul. Prosta 18, 00-950 Warszawa
68     ul. Prosta 18, 00-950 Warszawa
83     ul. Prosta 18, 00-950 Warszawa
121    ul. Prosta 18, 00-950 Warszawa
124    ul. Prosta 18, 00-950 Warszawa
131    ul. Prosta 18, 00-950 Warszawa
133    ul. Prosta 18, 00-950 Warszawa
134     ul. Grzybowskiej 78, 00-844 w
137    ul. Prosta 18, 00-950 Warszawa
141    ul. Prosta 18, 00-950 Warszawa
150    ul. Prosta 18, 00-950 Warszawa
231    ul. Prosta 18, 00-950 Warszawa
300     ul. Grzybowskiej 78, 00-844 w
327    ul. Prosta 18, 00-950 Warszawa
358    ul. Prosta 18, 00-950 Warszawa
374    ul. Prosta 18, 00-950 Warszawa
384    ul. Prosta 18, 00-950 Warszawa
399    ul. Prosta 18, 00-950 Warszawa
Name: informacje praktyczne, dtype: object

# KAPITAL_ZAKLADOWY_TOWARZYSTWA

In [264]:
capital_regex = re.compile('zakładowy:?.{,20}([\d\s,]+)', re.IGNORECASE)

In [265]:
def get_capital(text: str) -> str:
    regex = capital_regex.search(text)
    
    if regex is None:
        return None
    else:
        return regex.group(1)

In [266]:
capital = df_practial_info.apply(get_capital)

In [267]:
capital[~capital.isna()]

9       612,00 
134     612,00 
300     612,00 
Name: informacje praktyczne, dtype: object

# WALUTA_KAPITALU_ZAKLADOWEGO_TO

In [298]:
currency_regex = re.compile('(przedstawiono\sw\s)(\w{3})', re.IGNORECASE)

In [299]:
def get_currency(text: str) -> str:
    regex = currency_regex.search(text)
    if regex is None:
        return None
    else:
        return regex.group(2)   

In [300]:
currency = df_practial_info.apply(get_currency)

In [304]:
# currency[~currency.isna()]

# CZY_ESG

In [333]:
esg_regex = re.compile('esg' , re.IGNORECASE)
not_esg_regex = re.compile('.{0,10}nie.{0,10}esg.{0,10}nie.{0,10}')

In [329]:
def get_esg(text: str) -> str:
    regex = esg_regex.search(text)
    
    if regex is None:
        return False
    else:
        not_regex = not_esg_regex.search(text)
        if not_regex is None:
            return False
        return True

In [330]:
esg = df_practial_info.apply(get_esg)

In [331]:
esg[~esg.isna()]

11     Informacje praktyczne Amundi Stars Global Ecol...
60     Informacje praktyczne Amundi Stars Global Ecol...
228    westor ma prawo dokonywać zamiany jednostek uc...
241    y. Inwestor ma prawo dokonywać zamiany jednost...
265    Informacje praktyczne Amundi Stars Global Ecol...
Name: informacje praktyczne, dtype: object

In [332]:
esg.iloc[10]

'Informacje praktyczne Amundi Stars Global Ecology ESG Informacje praktyczne Niniejsze Kluczowe informacje dla inwestorów opisują Amundi Stars Global Ecol'

# TYP_FUNDUSZU
wyjątkowo `intro`

In [343]:
raw_text = df['raw_text']
raw_text = intro[~raw_text.isna()]

In [344]:
raw_text = raw_text.apply(split_text)

In [345]:
type_regex = re.compile('(S?FIO)')

In [346]:
def get_type(text: str) -> str:
    regex = type_regex.search(text)
    
    if regex is None:
        return None
    else:
        return regex.group(1)

In [347]:
types = raw_text.apply(get_type)

In [351]:
types[~types.isna()]

10     SFIO
22     SFIO
33     SFIO
34     SFIO
39     SFIO
       ... 
380    SFIO
382    SFIO
388    SFIO
394    SFIO
397    SFIO
Name: intro, Length: 64, dtype: object

# ISIN

In [354]:
isin_regex = re.compile('(\w{2}[\w\d]{9}\d)', re.IGNORECASE)

In [355]:
def get_isin(text: str) -> str:
    regex = isin_regex.search(text)
    
    if regex is None:
        return None
    else:
        return regex.group(1)    

In [356]:
isin = raw_text.apply(get_isin)

In [357]:
isin[~isin.isna()]

1      PLPPTFI00147
5      PLPPTFI00733
7      PLPPTFI00725
8      PLPPTFI00568
15     PLPPTFI00212
           ...     
361    PLPPTFI00014
365    PLPPTFI00691
366    PLPPTFI00600
375    PLUITFI00548
393    PLUITFI00506
Name: intro, Length: 76, dtype: object

# WYNIKI_OSIAGNIETE_W_PRZESZLOSCI

In [362]:
previous_scoring = df['wyniki osiągnięte w przeszłości']
previous_scoring.apply(split_text)

0      WYNIKI OSIĄGNIĘTE W PRZESZŁOŚCI Przedstawione ...
1      Wyniki osiągnięte w przeszłości Poniższy wykre...
2      WYNIKI OSIĄGNIĘTE W PRZESZŁOŚCI Subfundusz uru...
3      Wyniki osiągnięte w przeszłości Opłaty Depozyt...
4      Wyniki osiągnięte w przeszłości Jednorazowe op...
                             ...                        
396    Wyniki osiągnięte w przeszłości Opłaty Opłaty ...
397    Wyniki osiągnięte w przeszłości Wyniki history...
398    WYNIKI OSIĄGNIĘTE W PRZESZŁOŚCI Subfundusz uru...
399    Wyniki osiągnięte w przeszłości Jednostka typu...
400    Opłaty • Opłaty ponoszone przez Inwestora służ...
Name: wyniki osiągnięte w przeszłości, Length: 401, dtype: object

# DATA_PIERWSZEJ_WYCENY

In [363]:
previous_scoring_format = previous_scoring.apply(split_text)

In [426]:
first_date_regex = re.compile('((tworzon|ceni)).+(\d{4})\w?r?\.?')

In [427]:
def get_first_date(text: str) -> str:
    regex = first_date_regex.search(text)
    
    if regex is None:
        return None
    else:
        return regex.group(3)

In [428]:
dates = previous_scoring_format.apply(get_first_date)

In [429]:
dates[~dates.isna()]

1      2004
10     2020
15     2005
16     2022
42     2022
50     2019
56     2022
63     2022
81     2022
85     2022
87     2002
88     1996
92     2020
102    2010
111    1992
116    2022
119    2016
126    2003
130    2022
135    2022
140    2021
145    2021
152    2021
156    2011
166    2000
172    2021
173    2014
186    2021
188    2014
192    2021
199    1995
203    2020
204    2022
212    2020
213    2021
214    2020
216    2020
218    2020
222    2019
226    2020
229    2020
239    2021
246    2022
258    2017
274    2021
285    2016
286    2022
296    2022
314    2021
317    2021
324    2022
333    2022
335    2020
339    2022
353    2022
356    2020
361    2001
392    2022
Name: wyniki osiągnięte w przeszłości, dtype: object

# Offtop

In [1]:
def retrive_register_id(row) -> int:
    
    intro_text = row['intro']

    does_not_id_exist = (
        knf_df["nr_w_rejestrze"]
        .apply(lambda register_id: re.search(str(register_id), intro_text))
        .isna()
    )
    df_filtered = knf_df[~does_not_id_exist]

    if len(df_filtered["nr_w_rejestrze"].unique()) != 1:
        return None

    return df_filtered["nr_w_rejestrze"].iloc[0]

In [15]:
row = df.iloc[3]
intro_text = row['intro']

In [16]:
knf_df = pd.read_pickle('./../data/knf_funds.pkl')

In [25]:
knf_df

,fundusz,subfundusz,indentyfikator_krajowy,nr_w_rejestrze,towarzystwo,identyfikator_krajowy_funduszu
0,ABELIA CAPITAL Fundusz Inwestycyjny Zamknięty ...,None,PLFIZ000030,1212,AgioFunds Towarzystwo Funduszy Inwestycyjnych ...,PLTFI000036
1,AD ROYAL Fundusz Inwestycyjny Zamknięty Aktywó...,None,PLFIZ000572,420,AgioFunds Towarzystwo Funduszy Inwestycyjnych ...,PLTFI000036
2,AGIO Nieruchomości Komercyjnych Fundusz Inwest...,None,PLFIZ000037,986,AgioFunds Towarzystwo Funduszy Inwestycyjnych ...,PLTFI000036
3,AGIO PLUS Fundusz Inwestycyjny Otwarty,None,PLFIO000281,1309,AgioFunds Towarzystwo Funduszy Inwestycyjnych ...,PLTFI000036
4,AGIO PLUS Fundusz Inwestycyjny Otwarty,Subfundusz AGIO Akcji PLUS,PLFIO000264,1309,AgioFunds Towarzystwo Funduszy Inwestycyjnych ...,PLTFI000036
...,...,...,...,...,...,...
39,Universe 2 Niestandaryzowany Sekurytyzacyjny F...,None,PLFIZ000774,1079,White Berg Towarzystwo Funduszy Inwestycyjnych...,PLTFI000039
40,Universe 3 Niestandaryzowany Sekurytyzacyjny F...,None,PLFIZ000775,1078,White Berg Towarzystwo Funduszy Inwestycyjnych...,PLTFI000039
41,Universe Niestandaryzowany Sekurytyzacyjny Fun...,None,PLFIZ000777,1028,White Berg Towarzystwo Funduszy Inwestycyjnych...,PLTFI000039
42,Value Fund Poland Activist Fundusz Inwestycyjn...,None,PLFIZ000809,1421,White Berg Towarzystwo Funduszy Inwestycyjnych...,PLTFI000039


In [24]:
knf_df[knf_df['identyfikator_krajowy_funduszu'] == 'PLPPTFI00147']

,fundusz,subfundusz,indentyfikator_krajowy,nr_w_rejestrze,towarzystwo,identyfikator_krajowy_funduszu


In [19]:
intro_text

'PLPPTFI00147  \n \nKLUCZOWE INFORMACJE DLA INWESTORÓW \n \nNiniejszy dokument zawiera kluczowe informacje dla inwestorów dotyczące tego Subfunduszu. Nie są to materiały marketingowe. \nDostarczenie \ntych \ninformacji \njest \nwymogiem \nprawnym \nmającym \nna \ncelu \nułatwienie \nzrozumienia \ncharakteru  \ni ryzyka związanego z inwestowaniem w ten Subfundusz. Przeczytanie niniejszego dokumentu jest zalecane inwestorowi, aby mógł \non podjąć świadomą decyzję inwestycyjną. \n \n \n'

In [30]:
def get_nr(row):
    register_id = retrive_register_id(row)
    intro_text = row['intro']

    does_not_id_exist = (
        knf_df["indentyfikator_krajowy"]
        .apply(lambda x: re.search(str(x), intro_text))
        .isna()
    )
    df_filtered = knf_df[~does_not_id_exist]

    if df_filtered.shape[0] != 1:
        return 0
    else:
        return df_filtered["indentyfikator_krajowy"].iloc[0]



In [33]:
a = df.apply(get_nr)

TypeError: get_nr() got an unexpected keyword argument 'index'